In [1]:
import requests

In [2]:
url = 'https://www.cbr-xml-daily.ru/daily_json.js' # вынесем ссылку не ресурс в переменную

### Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

Сперва придумаем функцию, выполняющую условие. Либо через словарь либо через список кортежей

In [3]:
def max_rate(url):
    raw_data = requests.get(url).json()['Valute']
    currency_dict = {val['Name']:val['Value'] for val in raw_data.values()}
    result = {key for key,val in currency_dict.items() if val == max(currency_dict.values())}
    return list(result)[0]

In [4]:
def max_rate(url):
    raw_data = requests.get(url).json()['Valute']
    currency_dict = [(val['Name'], val['Value']) for val in raw_data.values()]
    result = max(currency_dict, key=lambda x: x[1])
    return result[0]

In [5]:
max_rate(url)

'СДР (специальные права заимствования)'

За основу возьмем класс из лекции и добавим в него функцию `max_rate` из задания

In [6]:
class Rate:
    """
    Добавление формата по-умалчанию каждому объекту класса
    По умалчанию краткий формат - выведет значение текущего курса.
    Если задать формат full - выведет полную информацию о валюте
    """
    def __init__(self, url, format_='value'):
        self.url = url
        self.format = format_
    
    def excange_data(self):
        """
        Загрузка сырых данных о всех курсах валют
        """
        self.raw_data = requests.get(self.url)
        return self.raw_data.json()['Valute']
    
    def report(self, curr):
        """
        Вывод отчета по какой-либо валюте по ее коду ЗАГЛАВНЫМИ буквами
        Если формат full - полную информацию
        Если формат value - только значение курса на последнюю дату
        Если код валюты неверный - выведется сообщение
        """
        response = self.excange_data()
        if curr in response:
            if self.format == 'value':
                return response[curr]['Value']
            elif self.format == 'full':
                return response[curr]
        else:
            return 'Нет такой валюты'
        
    def max_rate(self): # добавляем функцию
        """
        Если надо вывести название валюты с максимальным значением
        """
        response = self.excange_data()
        currency_dict = [(val['Name'], val['Value']) for val in response.values()]
        result = max(currency_dict, key=lambda x: x[1])
        return result[0]
        
    def eur(self):
        """частный метод для вывода информации по евро"""
        return self.report('EUR')
    def mdl(self):
        """частный метод для вывода информации по леям"""
        return self.report('MDL')
    def usd(self):
        """частный метод для вывода информации по долларам"""
        return self.report('USD')

In [7]:
obj_1 = Rate(url)

In [8]:
obj_1.max_rate()

'СДР (специальные права заимствования)'

In [9]:
obj_1.report('MDL')

43.2438

### Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

За основу возьмем класс `Rate`и сделаем новый, который унаследует признаки и методы

In [10]:
class RateDiff(Rate):
    """Класс наследует признаки родителя (Rate), 
    К нему добавляем признак diff
    По умолчанию diff=True - вместо значения курса выведет разницу между текущим и предыдущим
    """
    def __init__(self, url, format_='value', diff=True):
        super().__init__(url, format_)
        self.diff = diff
    
    def report(self, curr):
        """
        Вывод отчета по какой-либо валюте по ее коду ЗАГЛАВНЫМИ буквами
        Если формат full - полную информацию
        Если формат value - только значение курса на последнюю дату
        Если код валюты неверный - выведется сообщение
        Если во входных параметрах diff=True, то выведет разнице текущего и предыдущего курсов
        При формате full просто выведет информацию по валюте 
        """
        response = self.excange_data()
        if curr in response:
            if self.format == 'value':
                if self.diff == True:
                    return round((response[curr]['Value'] - response[curr]['Previous']), 4)
                else:
                    return response[curr]['Value']
            elif self.format == 'full':
                return response[curr]
        else:
            return 'Нет такой валюты'
        

In [11]:
obj_2 = RateDiff(url)

In [12]:
obj_2.report('EUR')

0.8187

In [13]:
obj_2.eur()

0.8187

In [14]:
obj_3 = RateDiff(url, format_='full')

In [15]:
obj_3.usd()

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 79.6274,
 'Previous': 79.1596}

### Задание 3
Напишите класс Designer, 
- Который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. 
- Получение международной премии – это +2 балла”). 
- Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

За основу берем класс `Employee`

In [16]:
class Employee:
    """Заводим парметры класса 
    имя и сеньёрити сотрудника
    По умлочанию грейд = 1"""
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority
        
        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

Создаем дочерний класс `Developer`, который:
- Наследует признаки родительского класса `Employee`
- При каждом запуске метода `check_if_it_is_time_for_upgrade` сеньёрити увеличивается на 1 пункт, дойдя до 5 пунктов прибавит 1 грейд
- Фиксирует апгрейд и возвращает в родительский класс на печать

In [17]:
class Developer(Employee):
    """
    Наследует признаки родительского класса Employee
    При каждом запуске метода check_if_it_is_time_for_upgrade сеньёрити увеличивается на 1 пункт, 
    дойдя до 5 пунктов прибавит 1 грейд
    Фиксирует апгрейд и возвращает в родительский класс на печать
    """
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [18]:
iwan = Developer('Иван', 0)

In [19]:
for i in range(20):
    iwan.check_if_it_is_time_for_upgrade()

Иван 1
Иван 1
Иван 1
Иван 1
Иван 2
Иван 2
Иван 2
Иван 2
Иван 2
Иван 3
Иван 3
Иван 3
Иван 3
Иван 3
Иван 4
Иван 4
Иван 4
Иван 4
Иван 4
Иван 5


Создаем дочерний класс `Designer`, который:
- Наследует признаки родительского класса `Employee`
- Добавляем собственный параметр `awd` - количество наград
- При каждом запуске метода `check_if_it_is_time_for_upgrade` сеньёрити увеличивается на 1 пункт, за награды добавляется 2 пункта. На количество наград запуск метода не влияет. Их количество прописывается в параметрах. Дойдя до 7 пунктов прибавит 1 грейд
- Фиксирует апгрейд и возвращает в родительский класс на печать

In [20]:
class Designer(Employee):
    """
    Наследует признаки родительского класса Employee
    Добавляем собственный параметр awd - количество наград
    При каждом запуске метода check_if_it_is_time_for_upgrade сеньёрити увеличивается на 1 пункт, 
    за награды добавляется 2 пункта. 
    На количество наград запуск метода не влияет. Их количество прописывается в параметрах. 
    Дойдя до 7 пунктов прибавит 1 грейд
    Фиксирует апгрейд и возвращает в родительский класс на печать
    """
    def __init__(self, name, seniority, awd):
        super().__init__(name, seniority)
        self.awd = awd
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все дизайнеры проходят аккредитацию
        self.seniority += 1
        # заводим счетчик общих пунктов как сумму сеньёрити + 2 пункта за награду
        self.total = self.awd * 2 + self.seniority        
        if self.total % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [21]:
stephan = Designer('Степан', 0, 2)

In [22]:
for i in range(20):
    stephan.check_if_it_is_time_for_upgrade()

Степан 1
Степан 1
Степан 2
Степан 2
Степан 2
Степан 2
Степан 2
Степан 2
Степан 2
Степан 3
Степан 3
Степан 3
Степан 3
Степан 3
Степан 3
Степан 3
Степан 4
Степан 4
Степан 4
Степан 4
